In [ ]:
#gestion de biblio
import sqlite3
from datetime import date

def get_connection():
    db = sqlite3.connect("biblio.db")
    return db

def create_table() : 
    db = get_connection()
    cur = db.cursor()

    cur.execute("""
        CREATE TABLE IF NOT EXISTS Livre(
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            titre TEXT NOT NULL,
            auteur TEXT,
            disponible INTEGER DEFAULT 1
        )
    """)

    cur.execute("""
        CREATE TABLE IF NOT EXISTS Adherent(
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            nom TEXT NOT NULL,
            email TEXT
        )
    """)

    cur.execute("""
        CREATE TABLE IF NOT EXISTS Emprunt(
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            id_livre INTEGER,
            id_adherent INTEGER,
            date_emprunt TEXT,
            date_retour TEXT,
            FOREIGN KEY(id_livre) REFERENCES Livre(id),
            FOREIGN KEY(id_adherent) REFERENCES Adherent(id)
        )
    """)

    db.commit()
    db.close()

In [ ]:
def ajouter_livre(titre, auteur):
    db = get_connection()
    cur = db.cursor()
    cur.execute("INSERT INTO Livre (titre, auteur, disponible) VALUES (?, ?, 1)", (titre, auteur))
    db.commit()
    db.close()

def supprimer_livre(id_livre):
    db = get_connection()
    cur = db.cursor()
    cur.execute("DELETE FROM Livre WHERE id = ?", (id_livre,))
    db.commit()
    db.close()

def lister_livres():
    db = get_connection()
    cur = db.cursor()
    cur.execute("SELECT * FROM Livre")
    livres = cur.fetchall()
    db.close()
    return livres

In [ ]:
def ajouter_adherent(nom, email):
    db = get_connection()
    cur = db.cursor()
    cur.execute("INSERT INTO Adherent (nom, email) VALUES (?, ?)", (nom, email))
    db.commit()
    db.close()

def lister_adherents():
    db = get_connection()
    cur = db.cursor()
    cur.execute("SELECT * FROM Adherent")
    adherents = cur.fetchall()
    db.close()
    return adherents


In [ ]:
def emprunter_livre(id_livre, id_adherent):

    db = get_connection()
    cur = db.cursor()

    cur.execute("SELECT disponible FROM Livre WHERE id = ?", (id_livre,))
    dispo = cur.fetchone()
    if dispo and dispo[0] == 1:
        cur.execute("""
            INSERT INTO Emprunt (id_livre, id_adherent, date_emprunt, date_retour)
            VALUES (?, ?, ?, NULL)
        """, (id_livre, id_adherent, str(date.today())))
        cur.execute("UPDATE Livre SET disponible = 0 WHERE id = ?", (id_livre,))
        db.commit()
    else:
        print("Ce livre n'est pas disponible pour l'emprunt.")

    db.close()

def retourner_livre(id_emprunt):

    db = get_connection()
    cur = db.cursor()

    cur.execute("SELECT id_livre FROM Emprunt WHERE id = ?", (id_emprunt,))
    result = cur.fetchone()
    if result:
        id_livre = result[0]
        cur.execute("UPDATE Emprunt SET date_retour = ? WHERE id = ?", (str(date.today()), id_emprunt))
        cur.execute("UPDATE Livre SET disponible = 1 WHERE id = ?", (id_livre,))
        db.commit()

    db.close()

def lister_emprunts():

    db = get_connection()
    cur = db.cursor()
    cur.execute("""
        SELECT E.id, L.titre, A.nom, E.date_emprunt, E.date_retour
        FROM Emprunt E
        JOIN Livre L ON E.id_livre = L.id
        JOIN Adherent A ON E.id_adherent = A.id
        ORDER BY E.date_emprunt DESC
    """)
    emprunts = cur.fetchall()
    db.close()
    return emprunts